# 1. 세팅

In [8]:
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import torch

# 2. 데이터셋 준비 

- 기법: 단어 인덱싱
- 데이터: 연습용 데이터, 2진 분류용

In [9]:
# 토크나이저와 어휘집 정의
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# 데이터 처리 파이프라인 정의
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

# DataLoader 준비
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    labels = torch.tensor(label_list, dtype=torch.int64)
    texts = torch.nn.utils.rnn.pad_sequence(text_list, padding_value=vocab["<pad>"])
    lengths = torch.tensor(lengths, dtype=torch.int64)
    return labels, texts, lengths

train_dataset = AG_NEWS(split='train')
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=False, collate_fn=collate_batch)

# 이제 train_dataloader를 이용하여 학습 루프에서 반복할 수 있습니다.


# 4. 모델 정의

1. nn.Embedding

```python
nn.Embedding(num_embeddings, embedding_dim, padding_idx=None)

# num_embeddings: 사전에 있는 고유 단어의 수
# embedding_dim: 각 단어 인덱스에 대해 생성되는 임베딩 벡터의 차원
# padding_idx: 패딩 인덱스를 지정하면, 해당 인덱스에 대한 벡터는 모델이 학습하는 동안 업데이트되지 않고 0으로 유지
```

# 5. 모델 학습

Batch 1, Loss: 0.6993478536605835
Batch 2, Loss: 0.6983717083930969
Batch 3, Loss: 0.6256541013717651
Batch 4, Loss: 0.7896268367767334
Batch 5, Loss: 0.7258651256561279
